# Aplicando Word2Vec usando BlazingText

Word2Vec es un algoritmo popular que se utiliza para generar representaciones vectoriales densas de palabras en grandes corpus mediante el aprendizaje no supervisado. Estas representaciones son útiles para muchas tareas de procesamiento del lenguaje natural (NLP) como el análisis de sentimientos, el reconocimiento de entidades con nombre y la traducción automática.

Para el siguiente ejemplo usaremos como guia los siguientes articulos:
- Guia de ejemplos oficial de AWS. [link](https://github.com/aws/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_word2vec_subwords_text8/blazingtext_word2vec_subwords_text8.ipynb?short_path=27c95ac)
- Entrenamiento y evaluación de un modelo Word2Vec usando BlazingText en Sagemaker. [link](https://t-redactyl.io/blog/2020/09/training-and-evaluating-a-word2vec-model-using-blazingtext-in-sagemaker.html)
    

## Configuracion inicial

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
print(role)# Este es el rol que SageMaker usaría para acceder a los recursos de AWS (S3, CloudWatch) en su nombre

bucket = sess.default_bucket()  # Reemplácelo con el nombre de su bucket si no se creara uno por defecto
print(bucket)
prefix = "blazingtext/subwords"  # Reemplace con el prefijo con el cual desea almacenar los datos si es necesario

## Ingestion de datos

A continuación, descargamos un conjunto de datos de la web con los que queremos entrenar los vectores de palabras. BlazingText espera un solo archivo de texto preprocesado con tokens separados por espacios y cada línea del archivo debe contener una sola oración.

En este ejemplo, entrenemos los vectores en el conjunto de datos text8 (100 MB), que es una versión pequeña (ya preprocesada) de un respaldo de Wikipedia.

In [ ]:
# Descargamos
!wget http://mattmahoney.net/dc/text8.zip -O text8.gz

In [ ]:
# Descomprimimos
!gzip -d text8.gz -f

Una vez completada la descarga y descompresión de datos, debemos cargarlos en S3 para que SageMaker pueda consumirlos para ejecutar un trabajo de entrenamiento. Usaremos Python SDK para cargar estos dos archivos en el depósito y la ubicación del prefijo que hemos establecido anteriormente.

In [ ]:
train_channel = prefix + "/train"

sess.upload_data(path="text8", bucket=bucket, key_prefix=train_channel)

s3_train_data = "s3://{}/{}".format(bucket, train_channel)


A continuación, debemos configurar una ubicación de salida en S3, donde se descargará el artefacto del modelo. Estos artefactos también son el resultado del trabajo de entrenamiento del algoritmo.

In [ ]:
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)

## Configuración de entrenamiento

In [ ]:
region_name = boto3.Session().region_name

In [ ]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print("Using SageMaker BlazingText container: {} ({})".format(container, region_name))

In [ ]:
bt_model = sagemaker.estimator.Estimator(
    container,
    role,
    train_instance_count=1,
    train_instance_type="ml.c4.2xlarge",  # Se recomienda encarecidamente el uso de ml.p3.2xlarge para lograr la máxima velocidad y rentabilidad
    train_volume_size=30,
    train_max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

In [ ]:
bt_model.set_hyperparameters(
    mode="skipgram",
    epochs=5,
    min_count=5,
    sampling_threshold=0.0001,
    learning_rate=0.05,
    window_size=5,
    vector_dim=100,
    negative_samples=5,
    subwords=True,
    min_char=3,
    max_char=6,
    batch_size=11,
    evaluation=True,
)

Ahora que los hiperparámetros están configurados, preparemos la configuracion entre nuestros canales de datos y el algoritmo. Para hacer esto, necesitamos crear los objetos sagemaker.session.s3_input desde nuestros canales de datos. Luego, estos objetos se colocan en un diccionario simple, que consume el algoritmo.

In [ ]:
train_data = sagemaker.session.s3_input(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="text/plain",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data}

Una vez finalizado el trabajo, se imprimirá el mensaje "Trabajo completado". El modelo entrenado se puede encontrar en el depósito de S3 que se configuró como output_path en el estimador.

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Hospedaje / Inferencia

Una vez finalizada la capacitación, podemos implementar el modelo entrenado como un punto de enlace alojado en tiempo real de Amazon SageMaker. Esto nos permitirá hacer predicciones (o inferencias) a partir del modelo. Tenga en cuenta que no tenemos que alojar en el mismo tipo de instancia que solíamos entrenar. Debido a que los puntos finales de instancia estarán en funcionamiento durante mucho tiempo, es recomendable elegir una instancia más barata para la inferencia.

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

### Usar formato JSON para inferencia

In [ ]:
words = ["awesome", "hello"]

payload = {"instances": words}

response = bt_endpoint.predict(json.dumps(payload), initial_args={'ContentType': 'application/json'})
vecs = json.loads(response)
print(vecs)

## Visual

Descargamos nuestro modelo desde S3

In [ ]:
!aws s3 cp s3://sagemaker-us-east-1-{account_number}/blazingtext/subwords/output/blazingtext-{bucket-postfix}/output/model.tar.gz - | tar -xz

Instalamos la libreria gensim para manejar los vectores

In [ ]:
!pip install gensim

Importamos la clase KeyedVectors y cargamos los vectores a consultar

In [ ]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('vectors.txt', binary=False)

Realizamos operaciones con vectores

In [ ]:
word_vectors.most_similar("family")

In [ ]:
word_vectors.similarity('woman', 'man')

In [ ]:
word_vectors.distance("media", "media")

In [ ]:
 word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
word_vectors.doesnt_match("breakfast cereal dinner lunch".split())

Cargamos las librerias para la reduccion de dimencionalidad de vectores

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
from matplotlib import pylab

## Generamos una grafica con todos los vectores en 2 dimenciones

In [ ]:
num_points = 400

first_line = True
index_to_word = []
with open("vectors.txt","r") as f:
    for line_num, line in enumerate(f):
        if first_line:
            dim = int(line.strip().split()[1])
            word_vecs = np.zeros((num_points, dim), dtype=float)
            first_line = False
            continue
        line = line.strip()
        word = line.split()[0]
        vec = word_vecs[line_num-1]
        for index, vec_val in enumerate(line.split()[1:]):
            vec[index] = float(vec_val)
        index_to_word.append(word)
        if line_num >= num_points:
            break
word_vecs = normalize(word_vecs, copy=False, return_norm=False)

tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=10000)
two_d_embeddings = tsne.fit_transform(word_vecs[:num_points])
labels = index_to_word[:num_points]

def plot(embeddings, labels):
    pylab.figure(figsize=(20,20))
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                       ha='right', va='bottom')
    pylab.savefig('tsne_blazing_text_initial.png', bbox_inches='tight')

plot(two_d_embeddings, labels)